# IMPORTS

In [36]:
from collections import *
from functools import *
from itertools import *
import sys
import re
import math
import string
import bisect
from parse import compile
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path


# Part 1

In [39]:
connections = {
    (1, 0): "LJ|",
    (-1, 0): "F7|",
    (0, 1): "J7-",
    (0, -1): "FL-"
}
movements = {
        '|': "UD",
        '-': "LR",
        'L': "UR",
        'J': "UL",
        '7': "DL",
        'F': "DR",
        '.': "",
    }
directions = {
    "U": (-1, 0),
    "D": (1, 0),
    "R": (0, 1),
    "L": (0, -1),
}
def find_start(grid):
    R, C = len(grid), len(grid[0])
    for r, c in product(range(R), range(C)):
        if grid[r][c] == "S": return r, c
    return 0, 0
def find_loop(sr, sc, grid, vertices):
    res = 1
    r, c = sr, sc
    dr, dc = directions[movements[grid[sr][sc]][0]]
    pr, pc = sr, sc
    r += dr
    c += dc
    if grid[r][c] not in connections[(dr, dc)]: return -1
    found = True
    while found:
        res += 1
        vertices.append((r, c))
        found = False
        for heading in movements[grid[r][c]]:
            dr, dc = directions[heading]
            nr, nc = r + dr, c + dc
            if grid[nr][nc] not in connections[(dr, dc)]: return -1
            if (nr, nc) == (pr, pc): continue
            if (nr, nc) == (sr, sc): return res
            pr, pc = r, c
            r, c = nr, nc
            found = True
            break
    return -1
def path():
    res = 0
    p = Path(vertices)
    v_set = set(vertices)
    for r, c in product(range(R), range(C)):
        if (r, c) in v_set: continue
        if p.contains_point((r, c)): res += 1
    return res
with open("big.txt", "r") as f:
    data = f.read().splitlines()
    grid = [list(line) for line in data]
    R, C = len(grid), len(grid[0])
    sr, sc = find_start(grid)
    for pipe in "|-LJ7F":
        vertices = [(sr, sc)]
        grid[sr][sc] = pipe # try with this pipe
        num_nodes = find_loop(sr, sc, grid, vertices)
        if num_nodes > 0:
            part_1 = num_nodes // 2
            part_2 = path()
            print("part 1:", part_1)
            print("part 2:", part_2)
            break

part 1: 7030
part 2: 285


In [50]:
import sys
from collections import defaultdict, deque

grid = []
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    for line in data:
        line = line.strip()
        if line == "": continue
        grid.append(line)

start = None
start_adj = []
adj = defaultdict(list)
for i, row in enumerate(grid):
    for j, cell in enumerate(row):
        neighbors = []
        if cell == "|":
            neighbors = [(2*i-1, 2*j), (2*i+1, 2*j)]
        elif cell == "-":
            neighbors = [(2*i, 2*j-1), (2*i, 2*j+1)]
        elif cell == "L":
            neighbors = [(2*i-1, 2*j), (2*i, 2*j+1)]
        elif cell == "J":
            neighbors = [(2*i-1, 2*j), (2*i, 2*j-1)]
        elif cell == "7":
            neighbors = [(2*i+1, 2*j), (2*i, 2*j-1)]
        elif cell == "F":
            neighbors = [(2*i+1, 2*j), (2*i, 2*j+1)]
        elif cell == "S":
            start = (2*i, 2*j)
        for x, y in neighbors:
            if x >= 0 and x < 2*len(grid) and y >= 0 and y < 2*len(row):
                adj[(2*i, 2*j)].append((x, y))

for i, row in enumerate(grid):
    for j, cell in enumerate(row):
        xs = []
        if i > 0: xs.append(2*i-1)
        if i+1 < len(grid): xs.append(2*i+1)
        ys = []
        if j > 0: ys.append(2*j-1)
        if j+1 < len(row): ys.append(2*j+1)
        for nx in xs:
            adj[(nx, 2*j)].append((2*i, 2*j))
        for ny in ys:
            adj[(2*i, ny)].append((2*i, 2*j))
                
inv_start = []
indeg = defaultdict(int)
for vert in adj:
    for vert2 in adj[vert]:
        indeg[vert2] += 1
        if vert2 == start:
            inv_start.append(vert)
for vert in inv_start:
    if indeg[vert] > 0:
        adj[start].append(vert)

INF = 1000000000
dst = defaultdict(lambda: INF)
bfs_q = deque()
bfs_q.append(start)
dst[start] = 0
ans = (0, start)
inloop = set()
while len(bfs_q) > 0:
    curcell = bfs_q.popleft()
    inloop.add(curcell)
    # print(curcell, dst[curcell])
    for nxt in adj[curcell]:
        if dst[nxt] == INF:
            dst[nxt] = dst[curcell] + 1
            ans = max(ans, (dst[nxt], nxt))
            bfs_q.append(nxt)
# print(ans[0])
# print(inloop)

ans2 = 0
vis = set()
for i, row in enumerate(grid):
    for j, cell in enumerate(row):
        if (2*i, 2*j) in inloop or (2*i, 2*j) in vis:
            continue
        added_to_q = set()
        cur_q = deque()
        cur_q.append((2*i, 2*j))
        added_to_q.add((2*i, 2*j))
        enclosed = True
        while len(cur_q) > 0:
            cx, cy = cur_q.popleft()
            # print(i, j, cx, cy)
            for nx, ny in [(cx-1, cy), (cx+1, cy), (cx, cy-1), (cx, cy+1)]:
                # print(i, j, cx, cy, nx, ny, (nx, ny) in inloop, (nx, ny) in added_to_q)
                if (nx, ny) in inloop or (nx, ny) in added_to_q:
                    continue
                assert((nx, ny) not in vis)
                if nx < 0 or nx >= 2*len(grid) or ny < 0 or ny >= 2*len(row):
                    enclosed = False
                    continue
                cur_q.append((nx, ny))
                added_to_q.add((nx, ny))
        for c in added_to_q:
            if c[0] % 2 == 0 and c[1] % 2 == 0 and enclosed:
                print("HEY", c)
                ans2 += 1
            vis.add(c)
print(ans2)

HEY (164, 112)
HEY (198, 72)
HEY (116, 150)
HEY (134, 140)
HEY (146, 150)
HEY (180, 148)
HEY (84, 170)
HEY (156, 130)
HEY (80, 148)
HEY (204, 76)
HEY (148, 144)
HEY (144, 146)
HEY (132, 162)
HEY (144, 80)
HEY (208, 78)
HEY (138, 140)
HEY (150, 150)
HEY (134, 142)
HEY (196, 94)
HEY (124, 110)
HEY (76, 190)
HEY (112, 126)
HEY (148, 146)
HEY (144, 148)
HEY (78, 170)
HEY (176, 192)
HEY (164, 158)
HEY (138, 142)
HEY (150, 152)
HEY (134, 144)
HEY (166, 96)
HEY (130, 118)
HEY (148, 148)
HEY (132, 140)
HEY (144, 150)
HEY (154, 154)
HEY (140, 124)
HEY (72, 208)
HEY (138, 144)
HEY (134, 146)
HEY (202, 144)
HEY (204, 190)
HEY (114, 152)
HEY (148, 150)
HEY (132, 142)
HEY (144, 152)
HEY (160, 96)
HEY (186, 108)
HEY (202, 78)
HEY (150, 130)
HEY (138, 146)
HEY (134, 148)
HEY (192, 182)
HEY (136, 142)
HEY (148, 152)
HEY (132, 144)
HEY (122, 112)
HEY (176, 172)
HEY (150, 132)
HEY (164, 164)
HEY (138, 148)
HEY (108, 110)
HEY (202, 122)
HEY (74, 84)
HEY (136, 144)
HEY (132, 146)
HEY (118, 158)
HEY (138, 

# Part 2

In [ ]:
turn_on = compile("turn on {:d},{:d} through {:d},{:d}")
turn_off = compile("turn off {:d},{:d} through {:d},{:d}")
toggle = compile("toggle {:d},{:d} through {:d},{:d}")
with open('input.txt', 'r') as f:
    data = f.read().splitlines()
    lights = [[0] * 1000 for _ in range(1000)]
    for line in data:
        to, toff, tog = turn_on.parse(line), turn_off.parse(line), toggle.parse(line)
        if to is not None:
            r1, c1, r2, c2 = to.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 1
        elif toff is not None:
            r1, c1, r2, c2 = toff.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] = max(0, lights[r][c] - 1)
        else:
            r1, c1, r2, c2 = tog.fixed
            for r, c in product(range(r1, r2 + 1), range(c1, c2 + 1)):
                lights[r][c] += 2
    print(sum(map(sum, lights)))